First few cells are for creating the master dataset (i.e by merging all csv files collected) 


In [22]:
#imports

import pandas as pd
import glob
from tqdm import tqdm

In [23]:
#specifies all file names using glob liberary and prints their path

all_files = glob.glob('../csv_files/*.csv')
print(all_files)

['../csv_files/3508153662.csv', '../csv_files/2028054341.csv', '../csv_files/00004.csv', '../csv_files/3602107603.csv', '../csv_files/3726298824.csv', '../csv_files/1203654458.csv', '../csv_files/3815991990.csv', '../csv_files/7567441.csv', '../csv_files/00005.csv', '../csv_files/Dataset2.csv', '../csv_files/00001.csv', '../csv_files/3722881981.csv', '../csv_files/3449474271.csv', '../csv_files/00003.csv', '../csv_files/Dataset1.csv', '../csv_files/Sumedha1.csv', '../csv_files/864405729.csv', '../csv_files/3119164873.csv', '../csv_files/00002.csv', '../csv_files/Sumedha2.csv', '../csv_files/3658686332.csv', '../csv_files/2519810956.csv']


In [24]:
#Reads all files and inserts them into a list. Also accounts for inconsistencies 
# in the saving of the dataset performed by the juniors during data collection.

file_dfs = []

for i,file in tqdm(enumerate(all_files)):
    try:
        df = pd.read_csv(file , sep = '|')
        
    except:
        df = pd.read_csv(file)
    file_dfs.append(df)

        

22it [00:00, 24.13it/s]


In [25]:
#Merge and print length of full dataset

df = pd.concat(file_dfs , axis = 0 , ignore_index=True)
print(len(df))

52983


The following cells are for processing and selecting the right records in terms of experience and job roles.
First few cells deal with removal of useless columns



In [26]:
df.head()

,Unnamed: 0,Job Applicants:,Openings:,PG :,Posted:,UG :,experience,job_desc,key_skills,position,salary,Doctorate :
0,0,59,1.0,Post Graduation Not Required,30+ days ago,Any Graduate,2 - 5 years,Job description\n Technical Skills\nChartered...,"['Assurance', 'Business transformation', 'Fixe...",PPM041 - AM-Internal Audit,Not Disclosed,NaN
1,1,39,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,4 - 6 years,Job description\nRoles and Responsibility\nDel...,"['Hibernate', 'Core Java', 'Front End', 'Oracl...",Java Full Stack Developer,Not Disclosed,NaN
2,2,130,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,5 - 10 years,Job description\n.\nRoles and Responsibility\n...,"['Computer science', 'Data analysis', 'Analyti...",Sr. Data Scientist,Not Disclosed,NaN
3,3,42,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,10 - 15 years,Job description\n\n\nRole purpose\n\nAs a Prin...,"['Linux', 'Coding', 'Project management', 'Ana...",Principle Architect,Not Disclosed,NaN
4,4,102,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,3 - 8 years,Job description\n 3-8 years experience in das...,"['tableau', 'Reporting Analyst', 'SQL database...",Industry Consulting Prin. Consultant,Not Disclosed,NaN


In [27]:
#Drop Columns 

df.drop(['Unnamed: 0' , 'Job Applicants:' , "Openings:" , 'Posted:', 'PG :', "UG :" , 'salary'] , axis = 1 , inplace= True)
df.head()


,experience,job_desc,key_skills,position,Doctorate :
0,2 - 5 years,Job description\n Technical Skills\nChartered...,"['Assurance', 'Business transformation', 'Fixe...",PPM041 - AM-Internal Audit,NaN
1,4 - 6 years,Job description\nRoles and Responsibility\nDel...,"['Hibernate', 'Core Java', 'Front End', 'Oracl...",Java Full Stack Developer,NaN
2,5 - 10 years,Job description\n.\nRoles and Responsibility\n...,"['Computer science', 'Data analysis', 'Analyti...",Sr. Data Scientist,NaN
3,10 - 15 years,Job description\n\n\nRole purpose\n\nAs a Prin...,"['Linux', 'Coding', 'Project management', 'Ana...",Principle Architect,NaN
4,3 - 8 years,Job description\n 3-8 years experience in das...,"['tableau', 'Reporting Analyst', 'SQL database...",Industry Consulting Prin. Consultant,NaN


In [28]:
#Select records that start with 0 or 1 in the experience columns
#This records thus pertain to entry level jobs that require little to no experience 

df = df[df.experience.str.startswith('0') | df.experience.str.startswith('1 ')]

In [29]:
len(df)

13451

In [30]:
#show distribution of experience requirements

df.experience.value_counts()

1 - 3 years     2057
0 - 2 years     1621
1 - 6 years     1553
0 - 1 years     1530
1 - 5 years     1338
1 - 4 years     1311
0 - 3 years     1133
1 - 2 years     1013
0 - 5 years      931
0 - 4 years      508
0 years          292
0 - 6 years       70
1 - 7 years       15
0 - 8 years       14
0 - 10 years      10
1 years           10
0 - 7 years        8
1 - 8 years        7
1 - 10 years       6
1 - 11 years       5
0 - 15 years       4
0 - 22 years       3
0 - 12 years       3
1 - 12 years       2
0 - 9 years        2
1 - 14 years       2
1 - 15 years       1
0 - 13 years       1
1 - 9 years        1
Name: experience, dtype: int64

In [31]:
#Processes the position column of the dataset. Strips spaces. 
#Filters out jobs that have a very broad range of experience requirements
#Print out the distribution of job roles

df.position = df.position.str.strip(' ')
df = df[~df['experience'].isin(['1 - 11 years' , '1 - 10 years' , '1 - 14 years' , '1 - 7 years' , '1 - 6 years' ,'0 - 5 years', '0 - 8 years' '1 - 8 years' , '1 - 5 years'])]
pd.set_option('display.max_rows' , 100)
df.position = df.position.str.lower()
df.position.value_counts()

php developer                         147
it recruiter                          130
business development executive        108
android developer                      86
executive - collections                78
                                     ... 
psl developer                           1
web developer intern                    1
php developer intern / trainee          1
flutter developer intern / trainee      1
sr full stack developer                 1
Name: position, Length: 4764, dtype: int64

In [32]:
#First line is used to select the job roles that we want to chose. We can alter this line to select job roles for different subjects (For example python)
#Filters out the dataset based on the roles we choose
roles = ['web designer' , 'web developer' , 'full stack developer' , 'react native developer' , 'angular developer' , 'wordpress developer' , 'full stack developer(angular 9,node js)' , 'react js developer' , 'php developers' , 'ui/ux designer' , 'node js developer' , 'front end developer']
df = df[df['position'].isin(roles)]
df.head()


,experience,job_desc,key_skills,position,Doctorate :
326,1 - 4 years,Job description\n\nURGENT HIRING FOR PHP DEVEL...,"['jQuery', 'Opencart', 'MySQL', 'Wordpress', '...",web developer,NaN
425,1 - 4 years,Job description\nRoles and Responsibilities\nH...,"['Drupal', 'Html5', 'Web Technologies', 'Javas...",web developer,NaN
589,1 - 3 years,Job description\nJob Description--\n\nJava Scr...,"['Computer science', 'CSS', 'jQuery', 'Front e...",front end developer,NaN
639,1 - 2 years,Job description\nResponsibilities\nDevelop and...,"['Prototype', 'Illustrator', 'Illustration', '...",ui/ux designer,NaN
680,1 - 3 years,Job description\nExperience in developing fron...,"['Css', 'Problem Solving', 'Javascript', 'Html']",angular developer,NaN


In [33]:
#Distribution of the final chosen job roles

df.position.value_counts()


web designer                               65
web developer                              50
wordpress developer                        35
angular developer                          29
full stack developer                       28
php developers                             18
react native developer                     17
front end developer                        12
ui/ux designer                             12
react js developer                         11
full stack developer(angular 9,node js)    10
node js developer                           8
Name: position, dtype: int64

In [34]:
#distribution of experience requirements of chosen records
df.experience.value_counts()

1 - 3 years    99
0 - 2 years    58
0 - 3 years    33
1 - 2 years    29
1 - 4 years    28
0 - 1 years    27
0 - 4 years    20
0 - 8 years     1
Name: experience, dtype: int64

In [35]:
#Save the dataset 
#Change the name for different subjects. 

df.to_excel('../dataset/Excel/web_dataset.xlsx' , engine  = 'xlsxwriter' , index = False)

In [36]:
len(df)

295